In [3]:
from sentence_transformers import SentenceTransformer
from tqdm.autonotebook import tqdm, trange

In [4]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)